In [147]:
import sqlite3
import pandas as pd

# Connection to database and sqlite version
try:
    sqliteconnect = sqlite3.connect('Hospitalinfo.db')
    cursor = sqliteconnect.cursor()
    print("Connection successful.")
    
    sqlite_select_query = 'select sqlite_version();'
    cursor.execute(sqlite_select_query)
    record = cursor.fetchall()
    print("Database version:", record)

except sqlite3.Error as error:
    print("Error occurred while connecting to sqlite", error)

Connection successful.
Database version: [('3.32.3',)]


In [140]:
# Creation of Doctor table
sqlite_create_table_query = '''CREATE TABLE "Doctor" (
	"DocID"	INTEGER,
	"DocName"	TEXT,
	"HospitalID"	INTEGER,
	"JoiningDate"	BLOB,
	"Specialty"	TEXT,
	"Exp"	TEXT,
	"salary"	real,
	PRIMARY KEY("DocID")
);'''
cursor.execute(sqlite_create_table_query)

#Populating data into the Doctor table

doc_data = [(101, 'David', 1, '2005-02-10', 'Pediatric', 40000),
(102, 'Michael', 1, '2018-07-23', 'Oncologist', 20000),
(103, 'Susan', 2, '2016-05-19', 'Garnacologist', 25000),
(104, 'Robert', 2, '2017-12-28', 'Pediatric', 28000),
(105, 'Linda', 3, '2004-06-04', 'Garnacologist', 42000),
(106, 'William', 3, '2012-09-11', 'Dermatologist', 30000),
(107, 'Richard', 4, '2014-08-21', 'Garnacologist', 32000),
(108, 'Karen', 4, '2011-10-17', 'Radiologist', 30000)]

cursor.executemany('''INSERT into Doctor 
(DocID, DocName, HospitalID, JoiningDate, Specialty, Exp, salary) 
VALUES (?, ?, ?, ?, ?, NULL, ?)''', doc_data)
cursor.fetchall()

[]

In [141]:
#Creating a dataframe from the SQL Doctor table

cursor.execute('''SELECT * FROM Doctor;''')
data = cursor.fetchall()
doctor_t = pd.DataFrame(data)
doctor_t.columns = [description[0] for description in cursor.description]
display(doctor_t)

,DocID,DocName,HospitalID,JoiningDate,Specialty,Exp,salary
0,101,David,1,2005-02-10,Pediatric,None,40000.0
1,102,Michael,1,2018-07-23,Oncologist,None,20000.0
2,103,Susan,2,2016-05-19,Garnacologist,None,25000.0
3,104,Robert,2,2017-12-28,Pediatric,None,28000.0
4,105,Linda,3,2004-06-04,Garnacologist,None,42000.0
5,106,William,3,2012-09-11,Dermatologist,None,30000.0
6,107,Richard,4,2014-08-21,Garnacologist,None,32000.0
7,108,Karen,4,2011-10-17,Radiologist,None,30000.0


In [124]:
# Creation of Hospital table
cursor.execute('''CREATE TABLE "Hospital" (
	"ID"	INTEGER,
	"Name"	TEXT,
	"BedCount"	INTEGER,
	PRIMARY KEY("ID")
);''')

# Populating the Hospital table with data using executemany
hosp_data = [(1, 'Mayo Clinic', 200),
(2, 'Cleveland Clinic', 400),
(3, 'John Hopkins', 1000),
(4, 'UCLA Medical Center', 1500)]

cursor.executemany('''INSERT into Hospital 
(ID, Name, Bedcount) 
VALUES
(?, ?, ?)
;''', hosp_data)

In [125]:
#Creating a dataframe from the SQL Hospital table
cursor.execute('''SELECT * FROM Hospital;''')
hospital_t = pd.DataFrame(cursor.fetchall())
hospital_t.columns = [description[0] for description in cursor.description]
display(hospital_t)

,ID,Name,BedCount
0,1,Mayo Clinic,200
1,2,Cleveland Clinic,400
2,3,John Hopkins,1000
3,4,UCLA Medical Center,1500


In [126]:
# Executing a join of both Hospital and Doctor tables
cursor.execute(''' SELECT * FROM Hospital, Doctor
WHERE Hospital.ID = Doctor.HospitalID
;''')
doc_hosp_join = pd.DataFrame(cursor.fetchall())
doc_hosp_join.columns = [description[0] for description in cursor.description]
display(doc_hosp_join)

,ID,Name,BedCount,DocID,DocName,HospitalID,JoiningDate,Specialty,Exp,salary
0,1,Mayo Clinic,200,101,David,1,2005-02-10,Pediatric,None,40000.0
1,1,Mayo Clinic,200,102,Michael,1,2018-07-23,Oncologist,None,20000.0
2,2,Cleveland Clinic,400,103,Susan,2,2016-05-19,Garnacologist,None,25000.0
3,2,Cleveland Clinic,400,104,Robert,2,2017-12-28,Pediatric,None,28000.0
4,3,John Hopkins,1000,105,Linda,3,2004-06-04,Garnacologist,None,42000.0
5,3,John Hopkins,1000,106,William,3,2012-09-11,Dermatologist,None,30000.0
6,4,UCLA Medical Center,1500,107,Richard,4,2014-08-21,Garnacologist,None,32000.0
7,4,UCLA Medical Center,1500,108,Karen,4,2011-10-17,Radiologist,None,30000.0


In [127]:
# Doctor names grouped by specialty and ordered by salary

cursor.execute(''' SELECT DocName, salary, Specialty FROM Doctor
GROUP BY Specialty, salary ORDER BY salary
;''')

cursor.fetchall()

[('Michael', 20000.0, 'Oncologist'),
 ('Susan', 25000.0, 'Garnacologist'),
 ('Robert', 28000.0, 'Pediatric'),
 ('William', 30000.0, 'Dermatologist'),
 ('Karen', 30000.0, 'Radiologist'),
 ('Richard', 32000.0, 'Garnacologist'),
 ('David', 40000.0, 'Pediatric'),
 ('Linda', 42000.0, 'Garnacologist')]

In [128]:
# Doctors by hospital
cursor.execute(''' SELECT DocName, Name FROM Doctor, Hospital
WHERE Hospital.ID = Doctor.HospitalID AND Name = 'John Hopkins'
;''')

cursor.fetchall()

[('Linda', 'John Hopkins'), ('William', 'John Hopkins')]

In [185]:
# Updating experience records for each row
update_exp = [(10, 101),
             (5, 102),
             (11, 103),
             (12, 104),
             (15, 105),
             (9, 106),
             (11, 107),
             (7, 108)]

cursor.executemany(''' UPDATE Doctor SET Exp = ? WHERE DocID = ?;''', update_exp)
sqliteconnect.commit()



print(cursor.fetchall())

[]


In [186]:
cursor.execute('''SELECT * FROM Doctor;''')
cursor.fetchall()

[(101, 'David', 1, '2005-02-10', 'Pediatric', '10', 40000.0),
 (102, 'Michael', 1, '2018-07-23', 'Oncologist', '5', 20000.0),
 (103, 'Susan', 2, '2016-05-19', 'Garnacologist', '11', 25000.0),
 (104, 'Robert', 2, '2017-12-28', 'Pediatric', '12', 28000.0),
 (105, 'Linda', 3, '2004-06-04', 'Garnacologist', '15', 42000.0),
 (106, 'William', 3, '2012-09-11', 'Dermatologist', '9', 30000.0),
 (107, 'Richard', 4, '2014-08-21', 'Garnacologist', '11', 32000.0),
 (108, 'Karen', 4, '2011-10-17', 'Radiologist', '7', 30000.0)]

In [187]:
sqliteconnect.close()